# Modelling

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.experimental import enable_iterative_imputer
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, IsolationForest
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, mean_squared_error, make_scorer, r2_score, mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.linear_model import BayesianRidge, Ridge, Lasso, LogisticRegression
from sklearn.feature_selection import VarianceThreshold, mutual_info_regression
from sklearn.cluster import KMeans, DBSCAN
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, accuracy_score
from statsmodels.stats.outliers_influence import variance_inflation_factor

## Read data

In [ ]:
# Read data
file_path = "../Data_Sources/Data_Cleaned/Table_for_modelling"
df = pd.read_csv(file_path)

## Feature Engineering

In [ ]:
def engineer_advanced_features(df):
    # Create interaction features for strong predictors
    df['recreational_total'] = df['Recreatief NL'] + df['Recreatief Buitenland']
    df['education_total'] = df['VO'] + df['Student'] + df['PO']
    
    # Create seasonal features
    df['Date'] = pd.to_datetime(df['Date'])
    df['season'] = df['Date'].dt.month.map({12:1, 1:1, 2:1,  # Winter
                                           3:2, 4:2, 5:2,    # Spring
                                           6:3, 7:3, 8:3,    # Summer
                                           9:4, 10:4, 11:4}) # Fall
    
    # Create weekend flag
    df['is_weekend'] = df['Date'].dt.dayofweek.isin([5, 6]).astype(int)
    
    # Combine holiday information
    df['is_any_holiday'] = ((df[['holiday_nl', 'holiday_be', 'holiday_de', 
                                'holiday_fr', 'holiday_gb', 'holiday_it']] == 1).any(axis=1)).astype(int)
    
    # Create tourism pressure indicator
    df['tourism_pressure'] = df['hotel_occupancy_index'] * df['tourism_season_strength']
    
    # lagged features for strong predictors
    for col in ['Recreatief NL', 'Recreatief Buitenland', 'VO', 'Student']:
        df[f'{col}_lag7'] = df[col].shift(7)
        df[f'{col}_lag14'] = df[col].shift(14)
    
    # rolling means for key metrics
    for col in ['nemo_market_share', 'hotel_occupancy_index', 'tourism_season_strength']:
        df[f'{col}_7day_avg'] = df[col].rolling(7).mean()
        df[f'{col}_30day_avg'] = df[col].rolling(30).mean()
    
    return df

In [ ]:
def enhance_weather_features(df):
    # Create weather condition categories
    df['is_good_weather'] = ((df['MeanTemp_C'] > 15) & 
                            (df['Precipitation_mm'] < 1) & 
                            (df['Sunshine_hours'] > 4)).astype(int)
    
    # Create extreme weather indicators
    df['is_rainy_day'] = (df['Precipitation_mm'] > 5).astype(int)
    df['is_very_hot'] = (df['MeanTemp_C'] > 25).astype(int)
    
    # Create weather-season interaction
    df['weather_season_score'] = df['MeanTemp_C'] * df['tourism_season_strength']
    
    return df

In [ ]:
def enhance_time_features(df):
    # Create holiday season flags
    df['is_summer_holiday'] = ((df['Date'].dt.month.isin([7, 8])) & (df['school_holiday'] == 1)).astype(int)
    
    # Create peak hours/days indicators
    df['is_peak_month'] = df['Date'].dt.month.isin([7, 8, 12]).astype(int)
    
    # Create cyclical features for month and day of week
    df['month_sin'] = np.sin(2 * np.pi * df['Date'].dt.month/12)
    df['month_cos'] = np.cos(2 * np.pi * df['Date'].dt.month/12)
    df['dayofweek_sin'] = np.sin(2 * np.pi * df['Date'].dt.dayofweek/7)
    df['dayofweek_cos'] = np.cos(2 * np.pi * df['Date'].dt.dayofweek/7)
    
    return df

In [ ]:
def preprocess_for_modeling(df):
    df = engineer_advanced_features(df)
    df = enhance_weather_features(df)
    df = enhance_time_features(df)
    
    # Handle missing values in new features
    df = df.fillna(method='ffill').fillna(method='bfill')
    
    return df

# Apply preprocessing
df = preprocess_for_modeling(df.copy())

## Feature Selection 

In [ ]:
def convert_date_to_numerical(X):
    X_processed = X.copy()
    
    # Convert date column to numerical features
    if 'Date' in X_processed.columns:
        X_processed['Date'] = pd.to_datetime(X_processed['Date'])
        X_processed['year'] = X_processed['Date'].dt.year
        X_processed['month'] = X_processed['Date'].dt.month
        X_processed['day'] = X_processed['Date'].dt.day
        X_processed['dayofweek'] = X_processed['Date'].dt.dayofweek
        X_processed = X_processed.drop('Date', axis=1)
    
    # Convert boolean columns to int
    bool_columns = X_processed.select_dtypes(include=['bool']).columns
    for col in bool_columns:
        X_processed[col] = X_processed[col].astype(int)
    
    # Get only numeric columns
    numeric_columns = X_processed.select_dtypes(include=['int64', 'float64']).columns
    X_processed = X_processed[numeric_columns]
    
    return X_processed

df_processed = prepare_data_for_mi(df)

### Correlation analysis

In [ ]:
correlations = df_processed.corr()['Total_Visitors'].drop('Total_Visitors')

fig, ax = plt.subplots(figsize=(20, 18))
correlations.sort_values().plot(kind='barh', ax=ax, color='blue')
ax.axvline(x=0.4, color='red', linestyle='--')
ax.axvline(x=-0.4, color='red', linestyle='--')
ax.set_title('Feature Correlation with Total Visitors')
ax.set_xlabel('Correlation Coefficient')
ax.set_ylabel('Features')

### Mutual information

Split into training/test sets

In [ ]:
X = df_processed.drop(columns=["Total_Visitors"])
y = df_processed["Total_Visitors"]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=40)

In [ ]:
mi_scores = mutual_info_regression(X_train, y_train)

# Get mi scores
mi_scores_series = pd.Series(mi_scores, index=X_train.columns).sort_values(ascending=False)

fig, ax = plt.subplots(figsize=(20, 18))
mi_scores_series.plot(kind='barh', ax=ax, color='blue')
ax.axvline(x=0.25, color='red', linestyle='--')
ax.set_title('Feature Mutual info scores')
ax.set_xlabel('Mutual info scores')
ax.set_ylabel('Features')

## Total_Visitor demand prediciton 

In [ ]:
model_column_mapping = {
    'Total_Visitors': {
        'target': 'Total_Visitors',
        'exclude_columns': [
            # Individual visitor types (since we're predicting their sum)
            'Extern', 'PO', 'Recreatief Buitenland', 'Recreatief NL', 
            'Student', 'VO',
            # Derived totals that include individual types
            'recreational_total', 'education_total',
            # Lag features of individual types (to avoid data leakage)
            'Recreatief NL_lag7', 'Recreatief NL_lag14',
            'Recreatief Buitenland_lag7', 'Recreatief Buitenland_lag14', 
            'VO_lag7', 'VO_lag14', 'Student_lag7', 'Student_lag14'
        ]
    },
    
    'Extern': {
        'target': 'Extern',
        'exclude_columns': [
            # Other visitor types and totals
            'PO', 'Recreatief Buitenland', 'Recreatief NL', 'Student', 'VO',
            'Total_Visitors', 'recreational_total', 'education_total',
            # All lag features of other types
            'Recreatief NL_lag7', 'Recreatief NL_lag14',
            'Recreatief Buitenland_lag7', 'Recreatief Buitenland_lag14', 
            'VO_lag7', 'VO_lag14', 'Student_lag7', 'Student_lag14'
        ]
    },
    
    'PO': {
        'target': 'PO',
        'exclude_columns': [
            'Extern', 'Recreatief Buitenland', 'Recreatief NL', 'Student', 'VO',
            'Total_Visitors', 'recreational_total', 'education_total',
            'Recreatief NL_lag7', 'Recreatief NL_lag14',
            'Recreatief Buitenland_lag7', 'Recreatief Buitenland_lag14', 
            'VO_lag7', 'VO_lag14', 'Student_lag7', 'Student_lag14'
        ]
    },
    
    'Recreatief Buitenland': {
        'target': 'Recreatief Buitenland',
        'exclude_columns': [
            'Extern', 'PO', 'Recreatief NL', 'Student', 'VO',
            'Total_Visitors', 'recreational_total', 'education_total',
            'Recreatief NL_lag7', 'Recreatief NL_lag14',
            'VO_lag7', 'VO_lag14', 'Student_lag7', 'Student_lag14'
            # Keep own lag features: 'Recreatief Buitenland_lag7', 'Recreatief Buitenland_lag14'
        ]
    },
    
    'Recreatief NL': {
        'target': 'Recreatief NL',
        'exclude_columns': [
            'Extern', 'PO', 'Recreatief Buitenland', 'Student', 'VO',
            'Total_Visitors', 'recreational_total', 'education_total',
            'Recreatief Buitenland_lag7', 'Recreatief Buitenland_lag14', 
            'VO_lag7', 'VO_lag14', 'Student_lag7', 'Student_lag14'
            # Keep own lag features: 'Recreatief NL_lag7', 'Recreatief NL_lag14'
        ]
    },
    
    'Student': {
        'target': 'Student',
        'exclude_columns': [
            'Extern', 'PO', 'Recreatief Buitenland', 'Recreatief NL', 'VO',
            'Total_Visitors', 'recreational_total', 'education_total',
            'Recreatief NL_lag7', 'Recreatief NL_lag14',
            'Recreatief Buitenland_lag7', 'Recreatief Buitenland_lag14', 
            'VO_lag7', 'VO_lag14'
            # Keep own lag features: 'Student_lag7', 'Student_lag14'
        ]
    },
    
    'VO': {
        'target': 'VO',
        'exclude_columns': [
            'Extern', 'PO', 'Recreatief Buitenland', 'Recreatief NL', 'Student',
            'Total_Visitors', 'recreational_total', 'education_total',
            'Recreatief NL_lag7', 'Recreatief NL_lag14',
            'Recreatief Buitenland_lag7', 'Recreatief Buitenland_lag14', 
            'Student_lag7', 'Student_lag14'
            # Keep own lag features: 'VO_lag7', 'VO_lag14'
        ]
    }
}

In [ ]:
X_total_visitors = df_processed.drop(columns=["Total_Visitors"] + model_column_mapping["Total_Visitors"]["exclude_columns"])
y_total_visitors = df_processed["Total_Visitors"]

X_train, X_test, y_train, y_test = train_test_split(X_total_visitors, y_total_visitors, random_state=40)

In [ ]:
# models 
models = [
    ("Ridge", Ridge(), [StandardScaler(), MinMaxScaler()]),
    ("Lasso", Lasso(), [StandardScaler(), MinMaxScaler()]),
    ("RandomForestRegressor", RandomForestRegressor(random_state=42), [None]),
    ("XGBRegressor", XGBRegressor(random_state=42), [StandardScaler(), MinMaxScaler()]),
]

# Parameter grids
param_grids = {
    "Ridge": {
        "model__alpha": np.logspace(-3, 3, 20)
    },
    "Lasso": {
        'model__alpha': [0.001, 0.01, 0.1, 1.0, 10.0],
        'model__max_iter': [5000, 10000],
        'model__tol': [1e-6, 1e-5],
        'model__selection': ['cyclic', 'random']
    },
    "RandomForestRegressor": {
        "model__n_estimators": [50, 100, 200],
        "model__max_depth": [None, 10, 20, 30]
    },
    "XGBRegressor": {
        "model__n_estimators": [100, 200, 500],
        "model__learning_rate": [0.01, 0.1, 0.2],
        "model__max_depth": [3, 4, 6],
        "model__min_child_weight": [1, 3],
        "model__subsample": [0.8, 0.9],
        "model__colsample_bytree": [0.8, 0.9]
    }
}

In [ ]:
def create_pipeline(model, scaler=None):
    """Create a pipeline with optional scaler and model"""
    if scaler is None:
        return Pipeline([('model', model)])
    else:
        return Pipeline([('scaler', scaler), ('model', model)])


def train_and_validate_models(X_train, X_test, y_train, y_test, cv_folds=5):
    """
    Train and validate all models with hyperparameter tuning
    
    Parameters:
    X_train, X_test: Training and testing features
    y_train, y_test: Training and testing targets
    cv_folds: Number of cross-validation folds
    
    Returns:
    Dictionary containing results for all models
    """
    
    results = {}
    best_models = {}
    
    print("Starting model training and validation...\n")
    print("="*80)
    
    for model_name, model, scalers in models:
        print(f"\nTraining {model_name}...")
        print("-" * 50)
        
        model_results = []
        
        # Try different scalers for each model
        for scaler in scalers:
            scaler_name = type(scaler).__name__ if scaler else "NoScaler"
            print(f"  Testing with {scaler_name}...")
            
            # Create pipeline
            pipeline = create_pipeline(model, scaler)
            
            # Get parameter grid for this model
            param_grid = param_grids.get(model_name, {})
            
            # Use RandomizedSearchCV for XGBoost due to large parameter space
            if model_name == "XGBRegressor":
                search = RandomizedSearchCV(
                    pipeline, 
                    param_grid, 
                    cv=cv_folds, 
                    scoring='neg_mean_squared_error',
                    n_iter=50,  # Limit iterations for XGBoost
                    random_state=42,
                    n_jobs=-1
                )
            else:
                search = GridSearchCV(
                    pipeline, 
                    param_grid, 
                    cv=cv_folds, 
                    scoring='neg_mean_squared_error',
                    n_jobs=-1
                )
            
            # Fit the search
            search.fit(X_train, y_train)
            
            # Make predictions
            y_pred_train = search.predict(X_train)
            y_pred_test = search.predict(X_test)
            
            # Calculate metrics
            train_mse = mean_squared_error(y_train, y_pred_train)
            test_mse = mean_squared_error(y_test, y_pred_test)
            train_r2 = r2_score(y_train, y_pred_train)
            test_r2 = r2_score(y_test, y_pred_test)
            train_mae = mean_absolute_error(y_train, y_pred_train)
            test_mae = mean_absolute_error(y_test, y_pred_test)
            
            # Cross-validation score
            cv_scores = cross_val_score(search.best_estimator_, X_train, y_train, 
                                      cv=cv_folds, scoring='neg_mean_squared_error')
            cv_rmse = np.sqrt(-cv_scores)
            
            result = {
                'model_name': model_name,
                'scaler': scaler_name,
                'best_params': search.best_params_,
                'best_cv_score': search.best_score_,
                'train_mse': train_mse,
                'test_mse': test_mse,
                'train_rmse': np.sqrt(train_mse),
                'test_rmse': np.sqrt(test_mse),
                'train_r2': train_r2,
                'test_r2': test_r2,
                'train_mae': train_mae,
                'test_mae': test_mae,
                'cv_rmse_mean': cv_rmse.mean(),
                'cv_rmse_std': cv_rmse.std(),
                'best_estimator': search.best_estimator_
            }
            
            model_results.append(result)
            
            print(f"    Best CV Score: {search.best_score_:.4f}")
            print(f"    Test RMSE: {np.sqrt(test_mse):.4f}")
            print(f"    Test R²: {test_r2:.4f}")
        
        # Find best scaler configuration for this model
        best_config = min(model_results, key=lambda x: x['test_rmse'])
        results[model_name] = model_results
        best_models[model_name] = best_config
        
        print(f"  Best configuration: {best_config['scaler']}")
        print(f"  Best Test RMSE: {best_config['test_rmse']:.4f}")
    
    return results, best_models

def print_detailed_results(results, best_models):
    """Print detailed results for all models"""
    
    print("\n" + "="*80)
    print("DETAILED RESULTS")
    print("="*80)
    
    # Create summary DataFrame
    summary_data = []
    for model_name, best_config in best_models.items():
        summary_data.append({
            'Model': model_name,
            'Best Scaler': best_config['scaler'],
            'Test RMSE': best_config['test_rmse'],
            'Test R²': best_config['test_r2'],
            'Test MAE': best_config['test_mae'],
            'CV RMSE (mean±std)': f"{best_config['cv_rmse_mean']:.4f}±{best_config['cv_rmse_std']:.4f}"
        })
    
    summary_df = pd.DataFrame(summary_data)
    summary_df = summary_df.sort_values('Test RMSE')
    
    print("\nMODEL COMPARISON SUMMARY:")
    print("-" * 30)
    print(summary_df.to_string(index=False))
    
    # Find overall best model
    best_overall = min(best_models.values(), key=lambda x: x['test_rmse'])
    
    print(f"\n🏆 BEST OVERALL MODEL:")
    print("-" * 25)
    print(f"Model: {best_overall['model_name']}")
    print(f"Scaler: {best_overall['scaler']}")
    print(f"Test RMSE: {best_overall['test_rmse']:.4f}")
    print(f"Test R²: {best_overall['test_r2']:.4f}")
    print(f"Test MAE: {best_overall['test_mae']:.4f}")
    print(f"Best Parameters: {best_overall['best_params']}")
    
    return best_overall

def get_feature_importance(best_model, feature_names=None):
    """Extract feature importance from the best model if available"""
    
    model = best_model['best_estimator']
    
    # Get the actual model from pipeline
    if hasattr(model, 'named_steps'):
        actual_model = model.named_steps['model']
    else:
        actual_model = model
    
    if hasattr(actual_model, 'feature_importances_'):
        importances = actual_model.feature_importances_
        
        if feature_names is not None:
            feature_imp = pd.DataFrame({
                'feature': feature_names,
                'importance': importances
            }).sort_values('importance', ascending=False)
            
            print(f"\n📊 FEATURE IMPORTANCE ({best_model['model_name']}):")
            print("-" * 40)
            print(feature_imp.head(10).to_string(index=False))
            
            return feature_imp
    
    elif hasattr(actual_model, 'coef_'):
        coefficients = actual_model.coef_
        
        if feature_names is not None:
            feature_imp = pd.DataFrame({
                'feature': feature_names,
                'coefficient': coefficients,
                'abs_coefficient': np.abs(coefficients)
            }).sort_values('abs_coefficient', ascending=False)
            
            print(f"\n📊 FEATURE COEFFICIENTS ({best_model['model_name']}):")
            print("-" * 45)
            print(feature_imp.head(10).to_string(index=False))
            
            return feature_imp
    
    print(f"\nFeature importance not available for {best_model['model_name']}")
    return None

In [ ]:
# Train and validate models
results, best_models = train_and_validate_models(X_train, X_test, y_train, y_test)

# Print detailed results
best_overall = print_detailed_results(results, best_models)

# Access the best trained model
best_trained_model = best_overall['best_estimator']

DETAILED RESULTS

MODEL COMPARISON SUMMARY:

                Model    Best Scaler  Test RMSE  Test R²   Test MAE CV RMSE (mean±std)
         XGBRegressor   MinMaxScaler 659.760270 0.728583 495.026963   618.6914±53.8980
RandomForestRegressor       NoScaler 667.275439 0.722364 497.114843   635.7325±59.0288
                Ridge StandardScaler 872.830283 0.524966 661.598681   772.1244±64.4633
                Lasso   MinMaxScaler 880.415731 0.516673 668.878415   774.8332±65.2600

🏆 BEST OVERALL MODEL:

Model: XGBRegressor
Scaler: MinMaxScaler
Test RMSE: 659.7603
Test R²: 0.7286
Test MAE: 495.0270
Best Parameters: {'model__subsample': 0.8, 'model__n_estimators': 100, 'model__min_child_weight': 3, 'model__max_depth': 6, 'model__learning_rate': 0.1, 'model__colsample_bytree': 0.8}

## Visitor demand per type prediction

In [ ]:
for visitor_type in visitor_types:
    print("-"*50)
    print(visitor_type)
    X = df_processed.drop(columns=[visitor_type] + model_column_mapping[visitor_type]["exclude_columns"])
    y = df_processed[visitor_type]

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=40)

    # Train and validate models
    results, best_models = train_and_validate_models(X_train, X_test, y_train, y_test)

    # Print detailed results
    best_overall = print_detailed_results(results, best_models)

    # Access the best trained model
    best_trained_model = best_overall['best_estimator']
    
    

Extern
🏆 BEST OVERALL MODEL:
Model: Ridge
Scaler: StandardScaler
Test RMSE: 97.7306
Test R²: 0.0068
Test MAE: 22.8596
Best Parameters: {'model__alpha': 233.57214690901213}

================================================================================

PO
🏆 BEST OVERALL MODEL:
Model: RandomForestRegressor
Scaler: NoScaler
Test RMSE: 58.2827
Test R²: 0.5088
Test MAE: 41.4976
Best Parameters: {'model__max_depth': 10, 'model__n_estimators': 200}

================================================================================

Recreatief Buitenland
🏆 BEST OVERALL MODEL:
Model: XGBRegressor
Scaler: StandardScaler
Test RMSE: 254.5829
Test R²: 0.7323
Test MAE: 186.6319
Best Parameters: {'model__subsample': 0.9, 'model__n_estimators': 500, 'model__min_child_weight': 1, 'model__max_depth': 4, 'model__learning_rate': 0.1, 'model__colsample_bytree': 0.9}

================================================================================

Recreatief NL
🏆 BEST OVERALL MODEL:
Model: XGBRegressor
Scaler: StandardScaler
Test RMSE: 621.0990
Test R²: 0.6215
Test MAE: 418.0994
Best Parameters: {'model__subsample': 0.8, 'model__n_estimators': 500, 'model__min_child_weight': 3, 'model__max_depth': 6, 'model__learning_rate': 0.01, 'model__colsample_bytree': 0.9}

================================================================================

Student
🏆 BEST OVERALL MODEL:
Model: XGBRegressor
Scaler: StandardScaler
Test RMSE: 33.3323
Test R²: 0.4473
Test MAE: 26.0796
Best Parameters: {'model__subsample': 0.8, 'model__n_estimators': 500, 'model__min_child_weight': 1, 'model__max_depth': 3, 'model__learning_rate': 0.01, 'model__colsample_bytree': 0.9}

================================================================================

VO
🏆 BEST OVERALL MODEL:
Model: XGBRegressor
Scaler: StandardScaler
Test RMSE: 74.4476
Test R²: 0.2887
Test MAE: 51.0027
Best Parameters: {'model__subsample': 0.8, 'model__n_estimators': 500, 'model__min_child_weight': 1, 'model__max_depth': 4, 'model__learning_rate': 0.01, 'model__colsample_bytree': 0.9}

